In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
import cvxpy as cp

## Generate the random Dataset


In [2]:
# We first generate a random dataset with number of features (m = 10) and number of instances (n = 100)
# We also generate a random label vector y \in {-1,1}

n = 100 # Number of instances
m = 10  # Number of Features

X = np.random.rand(n,m)
y = np.random.rand(n) # n-dimensional vector
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m) # m-dimensional vector
print(y)
print(X)

[ 1 -1  1 -1 -1 -1  1  1 -1 -1 -1 -1 -1  1 -1 -1 -1  1  1  1 -1 -1 -1  1
 -1 -1 -1  1 -1 -1  1 -1 -1 -1 -1 -1  1 -1  1 -1 -1 -1 -1  1  1  1 -1 -1
  1 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1  1 -1 -1 -1  1  1 -1 -1 -1 -1 -1  1
  1 -1  1  1  1 -1  1  1 -1 -1  1  1  1  1  1  1  1 -1  1  1 -1  1  1  1
 -1 -1  1 -1]
[[0.63165729 0.56259315 0.88911601 0.99243595 0.97148293 0.42418679
  0.9618053  0.64594619 0.71626428 0.60537136]
 [0.31332431 0.98303431 0.6052717  0.85491642 0.19914815 0.59130496
  0.54753222 0.90284044 0.71468023 0.09258027]
 [0.45464133 0.1415053  0.55578822 0.37241692 0.81183935 0.2797956
  0.91916133 0.6970818  0.06208752 0.32051508]
 [0.70376647 0.52942961 0.69125709 0.60365735 0.58405237 0.1239638
  0.65152582 0.47784894 0.54981361 0.58706855]
 [0.55386265 0.84703028 0.79557686 0.96638507 0.97611291 0.77637023
  0.07002549 0.74345025 0.68793438 0.62568656]
 [0.46695015 0.97228329 0.67228182 0.33928301 0.01464147 0.03805836
  0.23502414 0.23625863 0.52670759 0.0498618 ]
 [0.05

## An Implementation of the Logistic Loss 


In [19]:
def LogisticLossNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    Z = 1e3 # To prevent overflow in exp
    eps = 1e-12 # For numerical stability in log computation
    
    f = 0.0
    (n, m) = X.shape
    y_pred = np.zeros(n)
    # Cost function
    for i in range(n):
        for j in range(m):
            y_pred[i] += w[j] * X[i][j] 
            
        f += np.log(1 + np.exp(-y[i] * y_pred[i] / Z) + eps)
        
    for j in range(m):
        f += (lam * w[j] * w[j]) / 2
    
    # Gradient
    g = np.zeros(m)
    for k in range(m):
        for i in range(n):
            g[k] += -1 / Z * y[i] * X[i][k] / (1 + np.exp(y_pred[i] * y[i] / Z))
            
        g[k] += lam * w[k]
    
    return [f, g]     

In [20]:
start = time.time()
[f,g] = LogisticLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 3.950590133666992
Function value = [1827.10474119]
Printing Gradient:
[0.11749169 0.15899    0.81681781 ... 0.99246212 0.85652192 0.366302  ]


## An Implementation of the Least Squares 


In [21]:
def LeastSquaresNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    f = 0.0
    (n, m) = X.shape
    y_pred = np.zeros(n)
    # Cost function
    for i in range(n):
        for j in range(m):
            y_pred[i] += w[j] * X[i][j] 
            
        f += (y[i] - y_pred[i]) ** 2
        
    for j in range(m):
        f += (lam * w[j] * w[j]) / 2
    
    # Gradient
    g = np.zeros(m)
    for k in range(m):
        for i in range(n):
            g[k] += 2 * (y_pred[i] - y[i]) * X[i][k]
            
        g[k] += lam * w[k]
        
    return [f, g]     

In [22]:
start = time.time()
[f,g] = LeastSquaresNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 2.1725127696990967
Function value = [6.4106057e+08]
Printing Gradient:
[255753.40689687 240648.72892564 254237.27395052 ... 238644.84294679
 263388.49241077 236782.33777121]


## An Implementation of the Hinge Loss 

In [23]:
def HingeLossNaive(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    f = 0.0
    (n, m) = X.shape
    y_pred = np.zeros(n)
    # Cost function
    for i in range(n):
        for j in range(m):
            y_pred[i] += w[j] * X[i][j] 
            
        f += max(0, 1 - y[i] * y_pred[i])
        
    for j in range(m):
        f += (lam * w[j] * w[j]) / 2
    
    # Gradient
    g = np.zeros(m)
    for k in range(m):
        for i in range(n):
            if y_pred[i]*y[i] <= 1:
                g[k] += -1 * y[i] * X[i][k]
            
        g[k] += lam * w[k]
    
    return [f, g]

In [24]:
start = time.time()
[f,g] = HingeLossNaive(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Time Taken = 2.0967795848846436
Function value = [125883.29747759]
Printing Gradient:
[25.54630495 25.78453339 24.58310184 ... 22.42755837 26.88608145
 21.21990544]


## Scalability of the code

In [25]:
n = 100
m = 10000

X = np.random.rand(n,m)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f,g] = LogisticLossNaive(w,X,y,1)
end = time.time()
print("Logistic Loss")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

start = time.time()
[f,g] = LeastSquaresNaive(w,X,y,1)
end = time.time()
print("Least Square")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

start = time.time()
[f,g] = HingeLossNaive(w,X,y,1)
end = time.time()
print("Hinge Loss")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Logistic Loss
Time Taken = 3.909250497817993
Function value = [1789.21708714]
Printing Gradient:
[0.18113387 0.05562184 0.75208868 ... 0.9953454  0.28985262 0.10854626]
Least Square
Time Taken = 2.183762550354004
Function value = [6.25283582e+08]
Printing Gradient:
[270274.16131549 249061.0909122  266718.70685752 ... 218919.31429342
 266244.22989347 262762.69786772]
Hinge Loss
Time Taken = 2.0598666667938232
Function value = [116701.93514227]
Printing Gradient:
[24.67905062 21.78016385 23.05061244 ... 20.77612319 23.42611931
 25.79488864]


## Implement a vectorized version 

In [26]:
def LogisticLossVec(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    Z = 1e3 # To prevent overflow in exp
    eps = 1e-12 # For numerical stability in log computation
    
    y_pred = X @ w
    f = np.sum(np.log(1 + np.exp(-y_pred * y / Z) + eps)) + lam * np.linalg.norm(w) / 2
    g = - X.T / Z @ (y / (1 + np.exp(y * y_pred / Z))) + lam * w
    return [f, g]     

In [27]:
def LeastSquaresVec(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    y_pred = X @ w
    f = np.sum((y_pred - y) ** 2) + lam * np.linalg.norm(w) / 2
    g = 2 * X.T @ (y_pred - y) + lam * w
    return [f, g]     

In [28]:
def HingeLossVec(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    y_pred = X @ w
    f = np.sum((1 >= y_pred * y) * (1 - y_pred*y)) + lam * np.linalg.norm(w) / 2
    g = - X.T @ ((1 >= y_pred * y) * y) + lam * w
    return [f, g]

In [29]:
n = 100
m = 10000

X = np.random.rand(n,m)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f,g] = LogisticLossVec(w,X,y,1)
end = time.time()
print("Logistic Loss")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

start = time.time()
[f,g] = LeastSquaresVec(w,X,y,1)
end = time.time()
print("Least Square")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

start = time.time()
[f,g] = HingeLossVec(w,X,y,1)
end = time.time()
print("Hinge Loss")
print("Time Taken = " + str(end - start))
print("Function value = " + str(f))
print("Printing Gradient:")
print(g)

Logistic Loss
Time Taken = 0.013798236846923828
Function value = 14020.012456035533
Printing Gradient:
[[0.34092635 0.34092635 0.34092635 ... 0.34092635 0.34092635 0.34092635]
 [0.57263392 0.57263392 0.57263392 ... 0.57263392 0.57263392 0.57263392]
 [0.81774459 0.81774459 0.81774459 ... 0.81774459 0.81774459 0.81774459]
 ...
 [0.32067814 0.32067814 0.32067814 ... 0.32067814 0.32067814 0.32067814]
 [0.46680414 0.46680414 0.46680414 ... 0.46680414 0.46680414 0.46680414]
 [0.90843705 0.90843705 0.90843705 ... 0.90843705 0.90843705 0.90843705]]
Least Square
Time Taken = 0.013409614562988281
Function value = 61980869772.81122
Printing Gradient:
[[247537.70437758 247537.70437758 247537.70437758 ... 247537.70437758
  247537.70437758 247537.70437758]
 [269971.68443648 269971.68443648 269971.68443648 ... 269971.68443648
  269971.68443648 269971.68443648]
 [268148.79633246 268148.79633246 268148.79633246 ... 268148.79633246
  268148.79633246 268148.79633246]
 ...
 [246460.20895865 246460.2089586

## Lets us code the above Loss Fuctions in CVXPY!

CVXPY is an open source Python-embedded modeling language for convex optimization problems. Link: https://www.cvxpy.org/

In [14]:
def LogisticLossCVXPY(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    Z = 1e3 # To prevent overflow in exp
    eps = 1e-12 # For numerical stability in log computation
    
    expression = cp.sum(cp.log(1 + cp.exp(cp.multiply(-y, X @ w / Z)) + eps)) + lam * cp.norm(w, 2) / 2
    Problem = cp.Problem(cp.Minimize(expression))
    f = expression.value
    g = - X.T / Z @ (y / (1 + np.exp(y * y_pred / Z))) + lam * w
    return [f, g]

In [15]:
def LeastSquaresCVXPY(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    expression = cp.sum_squares(X @ w - y) + lam * cp.norm(w, 2) / 2
    f = expression.value
    g = 2 * X.T @ (y_pred - y) + lam * w
    return [f, g]

In [16]:
def HingeLossCVXPY(w, X, y, lam):
    # Computes the cost function for all the training samples
    # where f is the function value and g is the gradient
    expression = cp.sum(cp.pos(1 - cp.multiply(y, X @ w))) + lam * cp.norm(w, 2) / 2
    Problem = cp.Problem(cp.Minimize(expression))
    f = expression.value
    return [f, g]

In [ ]:
import numpy as np
n = 100
m = 10

X = np.random.rand(n,m)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

start = time.time()
[f1,g1] = LogisticLossCVXPY(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value Naive = " + str(f1))
print("Printing Gradient Naive:")
print(g1)

start = time.time()
[f2,g2] = LeastSquaresCVXPY(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value For = " + str(f2))
print("Printing Gradient For:")
print(g2)

start = time.time()
[f2,g2] = HingeLossCVXPY(w,X,y,1)
end = time.time()
print("Time Taken = " + str(end - start))
print("Function value For = " + str(f2))
print("Printing Gradient For:")
print(g2)

## Compare the losses with Graph



In [ ]:
def LogisticLossFun(w, X, y, lam):
    return error_ll

def LeastSquaresFun(w, X, y, lam):
    return error_ls

def HingeLossFun(w, X, y, lam):
    return error_hl

def plot_errors(error_ll, error_ls, error_hl, num):
    plt.plot(num, error_ll, label="Logistic Loss")
    plt.plot(num, error_ls, label="Least Squares")
    plt.plot(num, error_hl, label="Hinge Loss")
    plt.show()
    return

In [ ]:
n = 100
m = 10000

X = np.random.rand(n,m)
y = np.random.rand(n)
ybin = [(int(yi >= 0.5) - int(yi < 0.5)) for yi in y]
y = np.array(ybin)
w = np.random.rand(m, 1)

error_ll = LogisticLossFun(w,X,y,1)
error_ls = LeastSquaresFun(w,X,y,1)
error_hl = HingeLossFun(w,X,y,1)
plot_errors(error_ll, error_ls, error_hl, 100)